In [301]:
import flappy_bird_env
import gymnasium
import gym
import torch
from gymnasium.utils.play import play
import numpy as np
import pygame
import math
from torch.autograd import Variable
import matplotlib.pyplot as plt

Характеристики среды:
    Имеется 2 действия. 1: сделать рывок вверх. 0: ничего не делать.
    Состояния: Изначально среда передавала изображение размерностью 800 * 576 * 3, но мы решили работать с 6 числами. Координаты и размеры трубы, и координаты птички.
    Ревард: +1 за каждую пройденную трубу и + 0.001 за каждый пройденный кадр без коллизии

In [282]:
env = gymnasium.make("FlappyBird-v0", render_mode="rgb_array")

In [283]:
env.reset()
env.render()

array([[[ 78, 192, 202],
        [ 78, 192, 202],
        [ 78, 192, 202],
        ...,
        [ 78, 192, 202],
        [ 78, 192, 202],
        [ 78, 192, 202]],

       [[ 78, 192, 202],
        [ 78, 192, 202],
        [ 78, 192, 202],
        ...,
        [ 78, 192, 202],
        [ 78, 192, 202],
        [ 78, 192, 202]],

       [[ 78, 192, 202],
        [ 78, 192, 202],
        [ 78, 192, 202],
        ...,
        [ 78, 192, 202],
        [ 78, 192, 202],
        [ 78, 192, 202]],

       ...,

       [[222, 216, 149],
        [222, 216, 149],
        [222, 216, 149],
        ...,
        [222, 216, 149],
        [222, 216, 149],
        [222, 216, 149]],

       [[222, 216, 149],
        [222, 216, 149],
        [222, 216, 149],
        ...,
        [222, 216, 149],
        [222, 216, 149],
        [222, 216, 149]],

       [[222, 216, 149],
        [222, 216, 149],
        [222, 216, 149],
        ...,
        [222, 216, 149],
        [222, 216, 149],
        [222, 216, 149]]

In [284]:
# play(env, keys_to_action={(pygame.K_SPACE,): np.array([1])}, noop=np.array([0]),
#      fps=24)

In [285]:
env.action_space

Discrete(2)

In [316]:
def gen_epsilon_greedy_policy(estimator, epsilon, n_action):
    def policy_function(state):
            probs = torch.ones(n_action) * epsilon / n_action
            q_values = estimator.predict(state)
            best_action = torch.argmax(q_values).item()
            probs[best_action] += 1.0 - epsilon
            action = torch.multinomial(probs, 1).item()
            return action 
    return policy_function

In [287]:
class LR_Estimate():
    def __init__(self, n_feat, n_state, n_action, lr=0.05):#n_state - размерность состояний теперь
        self.w, self.b = self.get_gaussian_wb(n_feat,n_state)
        self.n_feat = n_feat
        self.models = []
        self.optimizers = []
        self.criterion = torch.nn.MSELoss()
        for _ in range(n_action):
            model = torch.nn.Linear(n_feat,1)
            self.models.append(model)
            optimizer = torch.optim.SGD(model.parameters(), lr)
            self.optimizers.append(optimizer)

    def get_gaussian_wb(self, n_feat, n_state, sigma=.2):
        torch.manual_seed(0)
        w = torch.randn((n_state,n_feat))/sigma
        b = torch.rand((n_feat))* 2.0 * math.pi
        return w, b

    def get_feature(self, s):
        features = (2.0 / self.n_feat) ** 0.5 * torch.cos(torch.matmul(torch.tensor(s).float().T, self.w) + self.b)
        
        return features

    def update(self, s, a, y):
        features = Variable(self.get_feature(s))
        y_pred = self.models[a](features)

        loss = self.criterion(y_pred, Variable(torch.Tensor([y])))

        self.optimizers[a].zero_grad()
        loss.backward()
        self.optimizers[a].step()

    def predict(self,s):
        features = self.get_feature(s)
        # print(features)
        with torch.no_grad():
            m = [model(features) for model in self.models]
            b = torch.Tensor(len(m), len(m[0]))
            
            torch.cat(m, out=b)
            return b

In [288]:
estimator = LR_Estimate(10, 6, env.action_space.n)
policy = gen_epsilon_greedy_policy(estimator, 0.1, env.action_space.n)

In [289]:
env.action_space.n

2

In [290]:
print(policy)

<function gen_epsilon_greedy_policy.<locals>.policy_function at 0x7f8043531580>


In [291]:
def rgbtogray2(img):
    return np.array([[i[0] * 0.299 + i[1] * 0.587 + i[2] * 0.114 for i in j] for j in img]).astype(int)

In [292]:
s = np.zeros((200, 100, 3))
rgbtogray2(s).shape

(200, 100)

In [293]:
env.reset()
env.step(1)[4]

{'background': {'upper_left': (0, 0)},
 'pipes': [{'x': 695, 'height': 151, 'top': -489, 'bottom': 351}],
 'base': {'x1': -5, 'x2': 667, 'y': 700},
 'bird': {'x': 222, 'y': 365.0},
 'last_action': 0,
 'score': 0}

In [294]:
env.reset()[1]

{'background': {'upper_left': (0, 0)},
 'pipes': [{'x': 700, 'height': 314, 'top': -326, 'bottom': 514}],
 'base': {'x1': 0, 'x2': 672, 'y': 700},
 'bird': {'x': 222, 'y': 376},
 'last_action': 0,
 'score': 0}

In [295]:
# play(env, keys_to_action={(pygame.K_SPACE,): np.array([1])}, noop=np.array([0]),
# fps=24)

In [296]:
rewards = np.zeros(30)
for traj_num in range(30):
    info = env.reset()
    state = np.array(list(info[1]['pipes'][-1].values()) + list(info[1]['bird'].values()))
    is_done = False
    for ___ in range(100):
        if is_done:
            break
        action = policy(state)
        new_state, reward, is_done, __,  info = env.step(action)
        state = np.array(list(info['pipes'][-1].values()) + list(info['bird'].values()))
        rewards[traj_num] += reward 

/tmp/ipykernel_19126/3857906204.py:42: UserWarning: An output with one or more elements was resized since it had shape [2, 1], which does not match the required output shape [2]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at ../aten/src/ATen/native/Resize.cpp:26.)
  torch.cat(m, out=b)


In [297]:
mean_rew = sum(rewards) / len(rewards)
print(mean_rew)

0.060733333333333354


In [298]:
from scipy import stats

interval = stats.norm.interval(0.95, loc=mean_rew, scale=stats.tstd(rewards, axis=0))
print(interval)

(0.019335231582902714, 0.10213143508376399)


In [337]:
def q_learning(env, estimator, n_episode, epsilon, n_action, gamma=1.0, epsilon_decay=0.99):
    for episode in range(n_episode):
        policy = gen_epsilon_greedy_policy(estimator, epsilon * epsilon_decay ** episode, n_action)
        info = env.reset()
        state = np.array(list(info[1]['pipes'][-1].values()) + list(info[1]['bird'].values()))
        is_done = False
        while not is_done:
            action = policy(state)
            new_state, reward, is_done, __,  info = env.step(action)
            near_pipe = min(info['pipes'], key= lambda x: abs(x['x'] - info['bird']['x']))
            new_state = np.array(list(near_pipe.values()) + list(info['bird'].values()))
            # if len(info['pipes']) > 1 and abs(info['pipes'][0]['x'] - info['bird']['x']) > abs(info['pipes'][1]['x'] - info['bird']['x']):
                
            #     new_state = np.array(list(info['pipes'][1].values()) + list(info['bird'].values()))
            # else:
            #     new_state = np.array(list(info['pipes'][0].values()) + list(info['bird'].values()))
            q_values_next = estimator.predict(new_state)
            td_target = reward + gamma * torch.max(q_values_next )
            estimator.update(state, action, td_target)
            total_reward_episode[episode] += reward
        state = new_state

In [343]:
n_state = 6
n_action = env.action_space.n
n_feature = 5
lr = 0.03
estimator = LR_Estimate(n_feature, n_state, n_action, lr)

n_episode = 300
total_reward_episode = [0] * n_episode
q_learning(env, estimator, n_episode, 0.1, n_action)

/tmp/ipykernel_19126/3857906204.py:42: UserWarning: An output with one or more elements was resized since it had shape [2, 1], which does not match the required output shape [2]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at ../aten/src/ATen/native/Resize.cpp:26.)
  torch.cat(m, out=b)


In [342]:
print(max(total_reward_episode))

0.08900000000000007


In [344]:
plt.plot(total_reward_episode, 'b.')
plt.title('Зависимость вознаграждения в эпизоде от времени')
plt.xlabel('Эпизод')
plt.ylabel('Полное вознаграждение')
plt.show()